# Testing Tiny Imagenet data with LAP:

In [1]:
output_path = './outputs/graphs/'
results_path = 'outputs/tiny_imagenet_random_label/'

In [2]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import graph_code.graphing_utils as plotting

In [3]:
colwidth = 4.22716535
pagewidth = 8.7598425

In [4]:
results_list = []

for file in [
    f for f in os.listdir(results_path) if f.endswith("json")
]:
    with open(results_path + file) as f:
        results_temp = json.load(f)
    results_list.append(results_temp)


In [5]:
results = {}

for results_dict in results_list:
    
    for dataset in results_dict:
        if dataset not in results:
            results[dataset] = {}
        
        for corruption_type in results_dict[dataset]:
            if corruption_type not in results[dataset]:
                results[dataset][corruption_type] = {}

            for run in results_dict[dataset][corruption_type]:
                if run not in results[dataset][corruption_type]:
                    results[dataset][corruption_type][run] = {}

                for depression in results_dict[dataset][corruption_type][run]:

                    results[dataset][corruption_type][run][depression] = [
                        dict(epoch=int(epoch), **metrics) 
                        for epoch, metrics in results_dict[dataset][corruption_type][run][depression].items() 
                        if epoch != 'corrupt_sources'
                    ]

In [6]:
results_df = []

for dataset in results:
    for corruption_type in results[dataset]:
        for run in results[dataset][corruption_type]:
            for depression in results[dataset][corruption_type][run]:
                results_df.append(
                    pd.json_normalize(
                        results[dataset][corruption_type][run][depression]
                    )
                    .assign(
                        dataset=dataset,
                        corruption_type=corruption_type,
                        run=run,
                        depression=depression
                    )
                    .assign(epoch=lambda x: x['epoch']+1)
                )

results_df = pd.concat(results_df)
results_df = (
    results_df
    .replace({"depression": {"true": True, "false": False}})
    .astype(
        {
            "run": "int64",
            "depression": "bool",
        }
    )
)

In [7]:
results_df = results_df.melt(
    id_vars=['dataset', 'corruption_type', 'run', 'depression', 'epoch'],
    var_name='metric',
    value_name='value'
)

In [11]:
def bold_max_value_latex(x, model_names):
    x = x.copy()
    len_cols = x.shape[0]
    n_models = len(model_names)
    idx_bold = (
        x
        [-n_models:]
        .str.replace(" ", "")
        .str.split("±")
        .str[0]
        .astype(float)
        .argmax()
    )
    max_val = x.iloc[idx_bold+len_cols-n_models]
    max_val, max_std = map(float, max_val.split("±"))

    for i in range(len_cols-n_models, len_cols):
        val = x.iloc[i]
        val, std = map(float, val.split("±"))
        if val >= max_val - max_std and val <= max_val + max_std:
            x.iloc[i] = '\\textbf{' + x.iloc[i] + '}'
    
    return x


corruption_order = [
    "Original Data",
    "Random Label",
]


model_order = [
    'Standard',
    'LAP (Ours)',
]

results_final_df = (
    results_df
    .loc[lambda df: df.metric == 'test_top5acc']
    .groupby(["depression", "corruption_type", "run"])
    [['value']]
    .max()
    .assign(value = lambda x: x['value']*100)
    .reset_index()
    .groupby(["depression", "corruption_type"])
    ['value']
    .agg(['mean', 'std'])
    .assign(
        mean_std = lambda x: 
            np.round(x['mean'], 2).astype(str) 
            + " ± "
            + np.round(x['std'], 2).astype(str),
    )
    ['mean_std']
    .to_frame()
    .reset_index()
    .rename(columns={
        "depression": "LAP", 
        "corruption_type": "Noise Type",
        "mean_std": "Top-5 Accuracy"
    })
    .replace(
        {
            "LAP": {
                False: "Standard",
                True: "LAP (Ours)"
            },
            "Noise Type": {
                "random_label": "Random Label",
                "original": "Original Data",
            }
        }
    )
    .pivot(
        index="Noise Type",
        columns="LAP",
        values="Top-5 Accuracy"
    )
    .reset_index()
    [[
        'Noise Type',  'Standard', 'LAP (Ours)',
    ]]
    .sort_values(
        "Noise Type", key=lambda x: x.map(corruption_order.index)
    )
    # makes bold with latex:
    .apply(
        bold_max_value_latex,
        model_names = model_order,
        axis=1
    )
)

results_final_df

LAP,Noise Type,Standard,LAP (Ours)
0,Original Data,\textbf{61.32 ± 0.61},60.62 ± 0.71
1,Random Label,46.27 ± 1.04,\textbf{54.48 ± 0.57}


In [12]:
print(
    results_final_df.to_latex(index=False)
)

\begin{tabular}{lll}
\toprule
Noise Type & Standard & LAP (Ours) \\
\midrule
Original Data & \textbf{61.32 ± 0.61} & 60.62 ± 0.71 \\
Random Label & 46.27 ± 1.04 & \textbf{54.48 ± 0.57} \\
\bottomrule
\end{tabular}

